BVP

laplace equation
concentric balls around center (0, 0) - drichlet bc on both the surfaces
smallest eigenvalues and functions

In [1]:
import numpy as np
import ufl
from mpi4py import MPI
from petsc4py import PETSc
from slepc4py import SLEPc
import pyvista as pv
from dolfinx import mesh, fem, plot

#1 generate concentric balls domain of radius 5 and 1   drichlet drichlet 

In [2]:
inner_radius = 1.0
outer_radius = 5.0

In [3]:
import gmsh
from dolfinx.io import gmshio

def get_domain(inner_radius: float, outer_radius: float, filename: str = None):
    """
    inner_radius: float, 
    outer_radius: float
    """

    gmsh.initialize()
    model = gmsh.model()
    model.add("domain")
    model.setCurrent("domain")


    outer_disk = model.occ.addDisk(0, 0, 0, outer_radius,outer_radius, tag=1)
    inner_disk = model.occ.addDisk(2, 0, 0, inner_radius,inner_radius, tag=2)
    shell_dims_tags, _ = model.occ.cut( #perform boolean cut
        [(2, outer_disk)], #target disk
        [(2, inner_disk)]  # tool disk
    )
    model.occ.synchronize()
    model.addPhysicalGroup(2, [shell_dims_tags[0][1]], tag = 1) #remaining area
    

    #set mesh options and generate mesh
    gmsh.option.setNumber("Mesh.MeshSizeFactor", 0.1)
    # gmsh.option.setNumber("Mesh.MeshSizeMin", 0.1)
    # gmsh.option.setNumber("Mesh.MeshSizeMax", 0.1)
    model.geo.synchronize()
    model.mesh.generate(2)
    # gmsh.fltk.run()
    # gmsh.write("spherical_shell.msh")  # Export to file
    


    #get domain from mesh
    domain, cell_markers, facet_markers = gmshio.model_to_mesh(model=model, comm=MPI.COMM_WORLD, rank = 0, gdim = 2)
    # gmsh.write("Sphere.xmdf") #save mesh

    gmsh.finalize()
    return domain, cell_markers, facet_markers

For plotting domain!

In [4]:
# from dolfinx import plot
# import pyvista

# def plot_domain(domain):

#     pyvista.start_xvfb()
#     gdim = domain.topology.dim #dimensions of domain!

#     domain.topology.create_connectivity(gdim, gdim)
#     topology, cell_types, geometry = plot.vtk_mesh(domain, gdim)
#     grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
#     plotter = pyvista.Plotter()
#     plotter.add_mesh(grid)
#     plotter.view_xy()
#     if not pyvista.OFF_SCREEN:
#         plotter.show()
#     else:
#         figure = plotter.screenshot("fundamentals_mesh.png")



In [5]:
# from dolfinx import fem
# import ufl

# def calculate_volume(domain, cell_markers):

#     dx = ufl.Measure("dx", domain=domain, subdomain_data=cell_markers)

#     # Total volume
#     total_volume = fem.assemble_scalar(fem.form(1.0 * dx))
#     print(f"Total volume: {total_volume:.4f}")

#     # Subdomain volume (e.g., physical group 1)
#     subdomain_volume = fem.assemble_scalar(fem.form(1.0 * dx(1)))
#     print(f"Subdomain volume: {subdomain_volume:.4f}")


In [6]:
domain, cell_markers, facet_markers = get_domain(inner_radius, outer_radius)
# plot_domain(domain)

Info    : Meshing 1D...                                                                                                                        
Info    : [  0%] Meshing curve 2 (Ellipse)
Info    : [ 60%] Meshing curve 3 (Ellipse)
Info    : Done meshing 1D (Wall 0.000382977s, CPU 0s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.218738s, CPU 0.209992s)
Info    : 4608 nodes 9218 elements


In [7]:
# calculate_volume(domain, cell_markers)

#2 Setup function space and apply boundary conditions

In [8]:
def setup_fe_space(domain):
    """Create function space and variational forms. Defining bilinear and linear forms."""
    V = fem.functionspace(domain, ("Lagrange", 1)) #set function space

    u, v = ufl.TrialFunction(V), ufl.TestFunction(V)
    a = fem.form(ufl.inner(ufl.grad(u), ufl.grad(v))*ufl.dx)
    m = fem.form(ufl.inner(u, v)*ufl.dx)

    return V, a, m


In [9]:
V, a, m = setup_fe_space(domain)

preparing boundary conditions
Drichlet BC u = 0

In [10]:
from petsc4py import PETSc
def on_boundary(x):
    return np.isclose(np.sqrt((x[0])**2 + x[1]**2), outer_radius)

def get_bcs(V):
    boundary_dofs = fem.locate_dofs_geometrical(V, on_boundary)
    bc = fem.dirichletbc(PETSc.ScalarType(0), boundary_dofs, V)

    return bc


#3 assemble matrices and apply boundary conditions

In [11]:
import dolfinx.fem.petsc

def assemble_and_apply_bc(V, a, m):

    #get bcs

    bc = get_bcs(V)

    #  Assemble matrices A and M

    A = fem.petsc.assemble_matrix(a, bcs=[bc]) #bc only on stiffness matrix
    A.assemble()
    M = fem.petsc.assemble_matrix(m) # mass matrix
    M.assemble()

    # null_vec = A.createVecLeft()   #for neumann problem
    # null_vec.set(1.0)
    # null_vec.normalize()
    # nullspace = PETSc.NullSpace().create(vectors=[null_vec])
    # A.setNullSpace(nullspace)

    return A, M



In [12]:
A, M = assemble_and_apply_bc(V, a, m)

In [13]:
print("A nnz:", A.getInfo())
print("M nnz:", M.getInfo())


A nnz: {'block_size': 1.0, 'nz_allocated': 31720.0, 'nz_used': 31720.0, 'nz_unneeded': 0.0, 'memory': 0.0, 'assemblies': 1.0, 'mallocs': 0.0, 'fill_ratio_given': 0.0, 'fill_ratio_needed': 0.0, 'factor_mallocs': 0.0}
M nnz: {'block_size': 1.0, 'nz_allocated': 31720.0, 'nz_used': 31720.0, 'nz_unneeded': 0.0, 'memory': 0.0, 'assemblies': 1.0, 'mallocs': 0.0, 'fill_ratio_given': 0.0, 'fill_ratio_needed': 0.0, 'factor_mallocs': 0.0}


In [14]:
print(A.norm())
print(M.norm())

248.35327896399866
0.6065066932100723


setup eigenvalue solver

In [15]:

def get_eigenvalues(A, M, n_eigen):

    eigensolver = SLEPc.EPS().create()
    eigensolver.setOperators(A, M)
    eigensolver.setProblemType(SLEPc.EPS.ProblemType.GHEP)
    # eigensolver.setDimensions(nev=n_eigen, ncv=2*n_eigen)  # Larger subspace
    eigensolver.setDimensions(nev=n_eigen)  # Larger subspace
    # eigensolver.setTolerances(tol=1e-8, max_it=1000)
    eigensolver.setWhichEigenpairs(SLEPc.EPS.Which.SMALLEST_REAL)

    eigensolver.setFromOptions()
    eigensolver.solve()
    n_conv = eigensolver.getConverged()

    if MPI.COMM_WORLD.rank == 0:
        print(f"Number of converged eigenvalues: {n_conv}")


    return eigensolver, n_conv

In [16]:
n_eigen = 5
eigensolver, n_conv = get_eigenvalues(A, M, n_eigen=n_eigen)

Number of converged eigenvalues: 6


In [17]:
import matplotlib.pyplot as plt
import numpy as np
import dolfinx.plot
from dolfinx.fem import Function, functionspace
from dolfinx.mesh import CellType, compute_midpoints, create_unit_cube, create_unit_square, meshtags
# import 
# Set some global options for all plots
transparent = False
figsize = 800


def plot_eigenfunctions(domain, V, eigensolver, A, nev_to_plot):
    """
    Plot the first few eigenfunctions for any given domain using matplotlib.
    Parameters:
        domain: dolfinx.mesh.Mesh
        V: dolfinx.fem.FunctionSpace
        eigensolver: SLEPc.EPS
        A: PETSc.Mat (needed for getVecs)
        n_conv: int (number of converged eigenpairs)
        nev_to_plot: int (number of eigenfunctions to plot)
    """
    if nev_to_plot == 0:
        return
    
    if MPI.COMM_WORLD.rank != 0:
        return

    topology, cell_types, geometry = plot.vtk_mesh(V)
    points = domain.geometry.x
    for i in range(nev_to_plot):
        eigval = eigensolver.getEigenvalue(i)
        error = eigensolver.computeError(i)

        print(f"Eigenvalue {i}: {eigval:.4f} (Error: {error:.4e})")

        r, _ = A.getVecs()
        eigensolver.getEigenvector(i, r)

        uh = fem.Function(V)
        uh.x.petsc_vec.setArray(r.array)
        uh.x.petsc_vec.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

        # Create PyVista grid
        values = uh.x.array.real
        # grid = pv.UnstructuredGrid(topology, cell_types, points)
        grid = pv.UnstructuredGrid(topology, cell_types, geometry)
        grid.point_data["u"] = values
        # The function "u" is set as the active scalar for the mesh, and
        # warp in z-direction is set
        grid.set_active_scalars("u")
        warped = grid.warp_by_scalar('u', factor=25)
        
        
        # Define the number of contour levels
        contour_levels = 25

        # Generate contours using the scalar data
        contours = grid.contour(contour_levels, scalars="u")

        # A plotting window is created with two sub-plots, one of the scalar
        # values and the other of the mesh is warped by the scalar values in
        # z-direction
        subplotter = pv.Plotter(shape=(1, 2))
        subplotter.subplot(0, 0)
        subplotter.add_text("Scalar contour field", font_size=14, color="black", position="upper_edge")
        subplotter.add_mesh(grid, show_scalar_bar=True, cmap="jet")
        subplotter.add_mesh(contours, color="blue", line_width=1)
        subplotter.view_xy()

        contours_warp = warped.contour(contour_levels, scalars="u")
        
        subplotter.subplot(0, 1)
        subplotter.add_text("Warped function", position="upper_edge", font_size=14, color="black")
        
        subplotter.add_mesh(warped, show_scalar_bar=True, cmap="jet")
        subplotter.add_mesh(contours_warp, color="blue", line_width=1)


        # if pyvista.OFF_SCREEN:
        #     subplotter.screenshot(
        #         "2D_function_warp.png",
        #         transparent_background=transparent,
        #         window_size=[figsize, figsize],
        #     )
        # else:
        # subplotter.link_views()
        subplotter.show()

        

# Usage example (after solving eigenproblem):
# plot_eigenfunctions(domain, V, eigensolver, A, n_conv, nev_to_plot=6)


In [18]:
plot_eigenfunctions(domain, V, eigensolver, A, n_conv)

Eigenvalue 0: 0.2371 (Error: 3.2805e-07)


Widget(value='<iframe src="http://localhost:40727/index.html?ui=P_0x77eb560316a0_0&reconnect=auto" class="pyvi…

Eigenvalue 1: 0.5306 (Error: 1.6766e-07)


Widget(value='<iframe src="http://localhost:40727/index.html?ui=P_0x77eb51d4e0d0_1&reconnect=auto" class="pyvi…

Eigenvalue 2: 0.6585 (Error: 1.1247e-07)


...

Widget(value='<iframe src="http://localhost:40727/index.html?ui=P_0x77eb51d4f250_3&reconnect=auto" class="pyvi…

Eigenvalue 4: 1.0322 (Error: 3.3943e-09)


Widget(value='<iframe src="http://localhost:40727/index.html?ui=P_0x77eb4bf98410_4&reconnect=auto" class="pyvi…

Eigenvalue 5: 1.1777 (Error: 1.4301e-08)


Widget(value='<iframe src="http://localhost:40727/index.html?ui=P_0x77eb4bf98550_5&reconnect=auto" class="pyvi…